In [1]:
import rasterio # for the extension to load
import xarray
import geopandas as gpd
from rasterio.windows import Window
from rasterio.plot import show

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def plot_raster_with_windows(raster_path, window_size=(512, 512)):
    with rasterio.open(raster_path) as src:
        num_cols = src.width
        num_rows = src.height

        for row_off in range(0, num_rows, window_size[0]):
            for col_off in range(0, num_cols, window_size[1]):
                # Define the window boundaries
                window = Window(col_off, row_off, min(window_size[1], num_cols - col_off),
                                min(window_size[0], num_rows - row_off))

                # Read the data in the window
                data = src.read(1, window=window)

                # Create the plot for the current window
                plt.imshow(data, cmap='viridis')
                plt.title(f"Window: {window.toslices()}")
                plt.colorbar()
                plt.show()

In [2]:
def plot_full_raster(raster_path):
    with rasterio.open(raster_path) as src:
        data = src.read(1)

        # Plot the full raster using rasterio.plot.show
        plt.figure(figsize=(10, 10))
        show(data, transform=src.transform, cmap='viridis')
        plt.title("Full Raster")
        plt.colorbar()
        plt.show()

In [ ]:
# Read the RiverAtlas Shapefile
dis_path = "../data/HydroRIVERS_v10_shp/raster/reduced_HydroRIVERS_v10.tif"

dis = gpd.read_file(dis_path)
df.head()

In [4]:
mountain_path = "../data/GlobalMountainsK3Binary/k3binary.tif"
mt_ds = rasterio.open(mountain_path)
mt_ds

<open DatasetReader name='../data/GlobalMountainsK3Binary/k3binary.tif' mode='r'>

In [ ]:
plt.imshow(mt_ds.read(1))
plt.show()

In [ ]:
raster_path = "../data/GlobalMountainsK3Binary/k3binary.tif"
plot_full_raster(raster_path)